In [150]:
from pathlib import Path 
import requests
#import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date
import pandas as pd



In [151]:

sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483*",
    security_token="1B6xNEvzIe5uroF6nRgKGTSk",
)

In [152]:
results = sf.query_all(
    """
    select id,Paciente__r.IdentificationId__pc, padre__c,Paciente__c,Paciente__r.Habilitado__c,Programas_por_Empresas__r.Datos_Empresa__c,datos2__c,  Programas_por_Empresas__c,Programas_por_Empresas__r.Name,  Empleador__c  from Paciente_Programas__c where Id = 'a1Y8c00000C5asnEAB' 
    """
)
derttyuiop = pd.DataFrame(results["records"])
derttyuiop.to_excel(
        "F:/Empresas SF/reportes/aaaaaa.xlsx", index=False
    )


In [153]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(7109, 8)

In [154]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6815, 5)

In [155]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c":"Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(7115, 10)

fin empresas y comienzo de rescate de pacientes


In [156]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1649891, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [157]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1210633, 4)

In [158]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1210633, 13)

In [160]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [161]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [162]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [163]:
#pacientes_empresas22.shape

In [164]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [165]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [166]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [182]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Socofar"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [183]:
Pacientes_Archivo.head(10)

,Empresa,Rut Titular,Dv Titular,Rut Asegurado,Dv,rut1,sexo,Parentesco,Descripcion,nombres,apellidos,nacimiento,Edad,email
0,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,5164267,8,5164267,8,5164267-8,M,NaN,NaN,ERNESTO,MALDONADO,1949-04-23,74.027778,NaN
1,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,6980755,0,6980755,0,6980755-0,M,NaN,NaN,ERNESTO,LARA,1954-12-05,68.411111,NaN
2,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7013000,9,7013000,9,7013000-9,M,NaN,NaN,OSCAR,SILVA,1960-12-02,62.419444,o.silva.steffens@gmail.com
3,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7044137,3,7044137,3,7044137-3,F,NaN,NaN,MIRNA,CASTRO,1959-10-25,63.522222,NaN
4,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7322151,K,7322151,K,7322151-K,F,NaN,NaN,MARIA,GONZALEZ,1955-11-19,67.455556,ginagonzacerd@gmail.com
5,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7544815,5,7544815,5,7544815-5,M,NaN,NaN,VICTOR,MARTINEZ,1954-10-28,68.513889,NaN
6,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7579156,9,7579156,9,7579156-9,F,NaN,NaN,RUTH,HUENUMAN,1959-07-28,63.763889,ruth.huenuman.123@gmail.com
7,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7776284,1,7776284,1,7776284-1,M,NaN,NaN,JORGE,VILLALOBOS,1960-08-11,62.727778,jorge1108@gmail.com
8,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7776284,1,8954075,5,8954075-5,F,CONYUGE,NaN,OLGA,MORENO,1991-01-09,32.319444,NaN
9,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7936265,4,7936265,4,7936265-4,F,NaN,NaN,MIRTHA,NEIRA,1960-05-30,62.925000,mirtha.neira@live.cl


In [184]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [185]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,Empresa,Rut Titular,Dv Titular,Rut Asegurado,Dv,rut1,sexo,Parentesco,Descripcion,nombres,apellidos,nacimiento,Edad,email,rut
0,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,5164267,8,5164267,8,5164267-8,M,NaN,NaN,ERNESTO,MALDONADO,1949-04-23,74.027778,NaN,51642678
1,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,6980755,0,6980755,0,6980755-0,M,NaN,NaN,ERNESTO,LARA,1954-12-05,68.411111,NaN,69807550
2,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7013000,9,7013000,9,7013000-9,M,NaN,NaN,OSCAR,SILVA,1960-12-02,62.419444,o.silva.steffens@gmail.com,70130009
3,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7044137,3,7044137,3,7044137-3,F,NaN,NaN,MIRNA,CASTRO,1959-10-25,63.522222,NaN,70441373
4,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7322151,K,7322151,K,7322151-K,F,NaN,NaN,MARIA,GONZALEZ,1955-11-19,67.455556,ginagonzacerd@gmail.com,7322151K


In [186]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19772\1949753364.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,Empresa,Rut Titular,Dv Titular,Rut Asegurado,Dv,rut1,sexo,Parentesco,Descripcion,nombres,apellidos,nacimiento,Edad,email,rut
0,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,5164267,8,5164267,8,5164267-8,M,NaN,NaN,ERNESTO,MALDONADO,1949-04-23,74.027778,NaN,51642678
1,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,6980755,0,6980755,0,6980755-0,M,NaN,NaN,ERNESTO,LARA,1954-12-05,68.411111,NaN,69807550
2,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7013000,9,7013000,9,7013000-9,M,NaN,NaN,OSCAR,SILVA,1960-12-02,62.419444,o.silva.steffens@gmail.com,70130009
3,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7044137,3,7044137,3,7044137-3,F,NaN,NaN,MIRNA,CASTRO,1959-10-25,63.522222,NaN,70441373
4,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7322151,K,7322151,K,7322151-K,F,NaN,NaN,MARIA,GONZALEZ,1955-11-19,67.455556,ginagonzacerd@gmail.com,7322151K


In [187]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
# Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19772\1486029825.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19772\1486029825.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [188]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(555, 25)

In [189]:
Pacientes_con_id.head()

,Empresa,Rut Titular,Dv Titular,Rut Asegurado,Dv,rut1,sexo,Parentesco,Descripcion,nombres,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,5164267,8,5164267,8,5164267-8,M,NaN,NaN,Ernesto,...,True,0018c00002egr6hAAA,51642678,Ernesto Samuel Maldonado Ulloa,RUT,None,None,None,Hombre,None
1,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,6980755,0,6980755,0,6980755-0,M,NaN,NaN,Ernesto,...,True,0018c00002eWtO4AAK,69807550,Ernesto Antonio Lara Fuentes,RUT,None,None,None,Hombre,None
2,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7013000,9,7013000,9,7013000-9,M,NaN,NaN,Oscar,...,True,0018c00002egqw0AAA,70130009,Oscar Arturo Silva Steffens,RUT,None,None,None,Hombre,None
3,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7044137,3,7044137,3,7044137-3,F,NaN,NaN,Mirna,...,True,0018c00002egr1TAAQ,70441373,Mirna Monica Castro Guajardo,RUT,None,None,None,Mujer,None
4,MEDICAL INTERNATIONAL LABORATORIES CORPORATION...,7322151,K,7322151,K,7322151-K,F,NaN,NaN,Maria,...,True,0018c00002egr1rAAA,7322151K,Maria Georgina Gonzalez Cerda,RUT,None,None,None,Mujer,None


In [190]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("Nan", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
8,False,Olga,Moreno,89540755,,1991-01-09,Mujer,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
11,False,Sandra,Gonzalez,102108426,,1988-05-06,Mujer,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
15,False,Cesar,Aguilera,25608421K,,2012-09-21,Hombre,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
18,False,Lidia,Munoz,80625340,,2012-02-25,Mujer,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
28,False,Zunilda,Villarroel,102972376,,1992-11-14,Mujer,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE


In [191]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [192]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
158723,a1Y8c00000C6VgWEAV,0018c00002amR8oAAE,a1W8c0000083iWpEAI,Socofar,0018c00002cXJTzAAO,Comercial Maicao SPA,Sin Holding Asociado,Socofar,Comercial Maicao SPA,Sin Póliza Asociada,None,EMP7405,A-0729
158724,a1Y8c00000C6VgXEAV,0018c00002amQoMAAU,a1W8c0000083iWpEAI,Socofar,0018c00002cXJTzAAO,Comercial Maicao SPA,Sin Holding Asociado,Socofar,Comercial Maicao SPA,Sin Póliza Asociada,None,EMP7405,A-0729
158725,a1Y8c00000C6VgYEAV,0018c00002amQoNAAU,a1W8c0000083iWpEAI,Socofar,0018c00002cXJTzAAO,Comercial Maicao SPA,Sin Holding Asociado,Socofar,Comercial Maicao SPA,Sin Póliza Asociada,None,EMP7405,A-0729
158726,a1Y8c00000C6VgZEAV,0018c00002amQoQAAU,a1W8c0000083iWpEAI,Socofar,0018c00002cXJTzAAO,Comercial Maicao SPA,Sin Holding Asociado,Socofar,Comercial Maicao SPA,Sin Póliza Asociada,None,EMP7405,A-0729
158727,a1Y8c00000C6VgbEAF,0018c00002amQozAAE,a1W8c0000083iWpEAI,Socofar,0018c00002cXJTzAAO,Comercial Maicao SPA,Sin Holding Asociado,Socofar,Comercial Maicao SPA,Sin Póliza Asociada,None,EMP7405,A-0729


In [193]:
campaña_seleccionada.shape

(3639, 13)

In [194]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [195]:
campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
campaña_en_pacientes = campaña_en_pacientes.rename(columns={'Programas_por_Empresas__c':'programas por empresa'})
campaña_en_pacientes['Paciente__c']=campaña_en_pacientes['Id Account Paciente']
campaña_en_pacientes['Programas_por_Empresas__c']=campaña_en_pacientes['programas por empresa']
campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )

In [181]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
